# Convert YOLO model to OpenVIN

## Installation

In [1]:
!git clone https://github.com/AlexeyAB/darknet.git
!ls darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14730, done.
remote: Total 14730 (delta 0), reused 0 (delta 0), pack-reused 14730
Receiving objects: 100% (14730/14730), 13.27 MiB | 6.30 MiB/s, done.
Resolving deltas: 100% (10020/10020), done.
3rdparty	DarknetConfig.cmake.in	include		       results
build		darknet_images.py	json_mjpeg_streams.sh  scripts
build.ps1	darknet.py		LICENSE		       src
build.sh	darknet_video.py	Makefile	       vcpkg.json
cfg		data			net_cam_v3.sh	       video_yolov3.sh
cmake		image_yolov3.sh		net_cam_v4.sh	       video_yolov4.sh
CMakeLists.txt	image_yolov4.sh		README.md


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [4]:
with open('darknet/Makefile', 'r') as file_:
  file_content = file_.read()
file_content = file_content.replace('GPU=0', 'GPU=1').replace('CUDNN=0', 'CUDNN=1').replace('OPENCV=0', 'OPENCV=1')

f = open('darknet/Makefile', 'w')
f.write(file_content)
f.close()

In [5]:
%cd darknet
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [6]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2021-03-18 07:26:35--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  24.1MB/s    in 1.4s    

2021-03-18 07:26:37 (24.1 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



## Convert YOLO model to Tensorflow frozen model

In [7]:
#clone a repo that helps with the conversion
!git clone https://github.com/mystic123/tensorflow-yolo-v3.git

Cloning into 'tensorflow-yolo-v3'...
remote: Enumerating objects: 219, done.
remote: Total 219 (delta 0), reused 0 (delta 0), pack-reused 219
Receiving objects: 100% (219/219), 57.63 KiB | 5.24 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [8]:
%cd tensorflow-yolo-v3/
!git checkout ed60b90

/content/darknet/tensorflow-yolo-v3
Note: checking out 'ed60b90'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at ed60b90 Merge pull request #53 from alexzhangxx/small_change


In [9]:
!python3 convert_weights_pb.py --class_names /content/darknet/data/coco.names --data_format NHWC --weights_file /content/darknet/yolov3-tiny.weights --tiny

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0318 07:26:42.164880 139966994327424 module_wrapper.py:139] From convert_weights_pb.py:38: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0318 07:26:42.174019 139966994327424 module_wrapper.py:139] From convert_weights_pb.py:40: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

Instructions for updating:
Please use `layer.__call__` method instead.
W0318 07:26:42.176979 139966994327424 deprecation.py:323] From /tensorflow-1.15.2/python3.7/tensorflow_core/contrib/layers/python/layers/layers.py:1057: Layer.apply (from tensorflow.python.keras.engine.

## Install OpenVINO 2021.2

In [10]:
%%time
%%capture
## install tools. Open Vino takes some time to download: 10-15 min sometimes.
!sudo apt-get install -y pciutils cpio
!sudo apt autoremove
## downnload installation files
!wget https://registrationcenter-download.intel.com/akdlm/irc_nas/17504/l_openvino_toolkit_p_2021.2.185.tgz
path = "l_openvino_toolkit_p_2021.2.185.tgz"
## install openvino
!tar xf "{path}"
%cd l_openvino_toolkit_p_2021.2.185/
!./install_openvino_dependencies.sh && \
    sed -i 's/decline/accept/g' silent.cfg && \
    ./install.sh --silent silent.cfg

CPU times: user 399 ms, sys: 85.3 ms, total: 484 ms
Wall time: 2min 36s


In [27]:
%cd /opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/install_prerequisites/
!./install_prerequisites_tf.sh

/opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/install_prerequisites
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubun

## Convert TF model to OpenVINO IR

In [11]:
%%writefile /content/darknet/yolo_v3_tiny.json
[
  {
    "id": "TFYOLOV3",
    "match_kind": "general",
    "custom_attributes": {
      "classes": 80,
      "anchors": [10, 14, 23, 27, 37, 58, 81, 82, 135, 169, 344, 319],
      "coords": 4,
      "num": 6,
      "masks": [[3, 4, 5], [0, 1, 2]],
      "entry_points": ["detector/yolo-v3-tiny/Reshape", "detector/yolo-v3-tiny/Reshape_4"]
    }
  }
]

Writing /content/darknet/yolo_v3_tiny.json


In [12]:
!cat /content/darknet/yolo_v3_tiny.json

[
  {
    "id": "TFYOLOV3",
    "match_kind": "general",
    "custom_attributes": {
      "classes": 80,
      "anchors": [10, 14, 23, 27, 37, 58, 81, 82, 135, 169, 344, 319],
      "coords": 4,
      "num": 6,
      "masks": [[3, 4, 5], [0, 1, 2]],
      "entry_points": ["detector/yolo-v3-tiny/Reshape", "detector/yolo-v3-tiny/Reshape_4"]
    }
  }
]

In [28]:
output_dir = '/content/yolov3_tiny'

!source /opt/intel/openvino_2021.2.185/bin/setupvars.sh && \
    python /opt/intel/openvino_2021.2.185/deployment_tools/model_optimizer/mo.py \
      --input_model /content/darknet/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb \
      --tensorflow_use_custom_operations_config /content/darknet/yolo_v3_tiny.json \
      --batch 1 \
      --data_type FP16 \
      --reverse_input_channel \
      --output_dir {output_dir}

[setupvars.sh] OpenVINO environment initialized
[ WARNING ]  Use of deprecated cli option --tensorflow_use_custom_operations_config detected. Option use in the following releases will be fatal. Please use --transformations_config cli option instead
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/content/darknet/tensorflow-yolo-v3/frozen_darknet_yolov3_model.pb
	- Path for generated IR: 	/content/yolov3_tiny
	- IR output name: 	frozen_darknet_yolov3_model
	- Log level: 	ERROR
	- Batch: 	1
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	True
TensorFlow specific parameters:


In [29]:
!ls -lah {output_dir}/frozen_darknet_yolov3_model.bin

-rw-r--r-- 1 root root 17M Mar 18 07:39 /content/yolov3_tiny/frozen_darknet_yolov3_model.bin


## Compile the IR model to a .blob for use on DepthAI modules/platform

In [30]:
import requests

url = "http://69.164.214.171:8083/compile"  # change if running against other URL

payload = {
    'compiler_params': '-ip U8 -VPU_NUMBER_OF_SHAVES 8 -VPU_NUMBER_OF_CMX_SLICES 8',
    'compile_type': 'myriad'
}
files = [
    ('definition', open(f'{output_dir}/frozen_darknet_yolov3_model.xml', 'rb')),
    ('weights', open(f'{output_dir}/frozen_darknet_yolov3_model.bin', 'rb'))
]
params = {
    'version': '2021.1',  # OpenVINO version, can be "2021.1", "2020.4", "2020.3", "2020.2", "2020.1", "2019.R3"
}

response = requests.post(url, data=payload, files=files, params=params)

with open(f'{output_dir}/model.blob', 'wb') as f:
  f.write(response.content)

In [31]:
!ls -lah {output_dir}/model.blob

-rw-r--r-- 1 root root 17M Mar 18 07:40 /content/yolov3_tiny/model.blob


In [32]:
from google.colab import files
files.download(f'{output_dir}/model.blob')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>